In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def read_excel(path):
    return pd.read_excel(path)

def imputing_data_row(df,missing):
    df_a = df[df['WAKTU'] == missing['WAKTU'] + pd.Timedelta(minutes=-15)]
    df_b = df[df['WAKTU'] == missing['WAKTU'] + pd.Timedelta(minutes=15)]
    MOTOR = -1
    MOBIL = -1
    TRUK = -1
    if(len(df_a) + len(df_b) == 2):
        result = pd.concat([df_a,df_b])
        MOTOR = round(result['MOTOR'].mean())
        MOBIL = round(result['MOBIL'].mean())
        TRUK = round(result['TRUK/BUS'].mean())
    return MOTOR, MOBIL, TRUK 

def imputing_data(df, HARI, JAM):
    df = df[(df['HARI'] == HARI) & (df['JAM'] == JAM)]
    
    # Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
    Q1 = df['SMP'].quantile(0.25)
    Q3 = df['SMP'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Menentukan batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Menemukan dan menghapus outlier
    df_cleaned = df[(df['SMP'] >= lower_bound) & (df['SMP'] <= upper_bound)]

    df_grouped = df_cleaned[['PEKAN', 'HARI', 'JAM', 'MOTOR', 'MOBIL','TRUK/BUS']]
    df_grouped = round(df_grouped.groupby(['PEKAN', 'HARI', 'JAM']).mean()).reset_index()

    return df_grouped

def perhitungan_jumlah_smp(df):
    df['SMP_MTR'] = df['MOTOR'] * 0.8
    df['SMP_MBL'] = df['MOBIL']
    df['SMP_TRK'] = df['TRUK/BUS'] * 2.5
    df['SMP'] = round(df['SMP_MTR'] + df['SMP_MBL'] + df['SMP_TRK'])
    df['JUMLAH'] = df['MOTOR'] + df['MOBIL'] + df['TRUK/BUS']
    return df

def menggabungkan_data(df, missing_times):
    # Menggabungkan data hilang dengan DataFrame utama
    df = pd.concat([df, missing_times], ignore_index=True)
    
    # Mengurutkan DataFrame berdasarkan kolom 'Waktu'
    df = df.sort_values(by=['TAHUN','BULAN','TANGGAL','HARI','JAM']).reset_index(drop=True)

    return df


In [3]:
path_1 = '../Data/Preprocessing/Data Balonggandu/2023/1_input_balonggandu_2023.xlsx'
path_2 = '../Data/Preprocessing/Data Balonggandu/2023/1_missing_times_balonggandu_2023.xlsx'

df = read_excel(path_1)
df_missing = read_excel(path_2)

df_missing.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-02 10:15:00,2023,1,1,2,Monday,10:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-02 22:45:00,2023,1,1,2,Monday,22:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-03 05:30:00,2023,1,1,3,Tuesday,05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-03 06:30:00,2023,1,1,3,Tuesday,06:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-04 15:30:00,2023,1,1,4,Wednesday,15:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for i, row in df_missing.iterrows():
    imputasi = imputing_data(df, row['HARI'], row['JAM'])
    break

In [5]:
imputasi

,PEKAN,HARI,JAM,MOTOR,MOBIL,TRUK/BUS
0,1,Monday,10:15,312.0,109.0,68.0
1,2,Monday,10:15,298.0,110.0,70.0
2,3,Monday,10:15,287.0,122.0,57.0
3,4,Monday,10:15,272.0,111.0,64.0
4,5,Monday,10:15,309.0,113.0,53.0


In [6]:
# Iterasi setiap baris dan mengisi nilai yang hilang
for i, missing in df_missing.iterrows():
    MOTOR, MOBIL, TRUK = imputing_data_row(df, missing)
    if (MOTOR != -1) & (np.isnan(missing['MOTOR'])):
        df_missing.at[i, 'MOTOR'] = MOTOR
        df_missing.at[i, 'MOBIL'] = MOBIL
        df_missing.at[i, 'TRUK/BUS'] = TRUK

In [7]:
# Iterasi setiap baris dan mengisi nilai yang hilang
for i, missing in df_missing.iterrows():
    imputasi = imputing_data(df, missing['HARI'], missing['JAM'])
    if (not imputasi.empty) & (np.isnan(missing['MOTOR'])):
        df_missing.at[i, 'MOTOR'] = imputasi['MOTOR'].values[0]
        df_missing.at[i, 'MOBIL'] = imputasi['MOBIL'].values[0]
        df_missing.at[i, 'TRUK/BUS'] = imputasi['TRUK/BUS'].values[0]

In [8]:
df_missing = perhitungan_jumlah_smp(df_missing)

In [9]:
df_missing.to_excel('../Data/Preprocessing/Data Balonggandu/2023/2_missing_times_balonggandu_2023.xlsx', index=False)

In [10]:
df_missing.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-02 10:15:00,2023,1,1,2,Monday,10:15,550.0,154.0,61.0,765.0,440.0,154.0,152.5,746.0
1,2023-01-02 22:45:00,2023,1,1,2,Monday,22:45,242.0,44.0,32.0,318.0,193.6,44.0,80.0,318.0
2,2023-01-03 05:30:00,2023,1,1,3,Tuesday,05:30,308.0,52.0,24.0,384.0,246.4,52.0,60.0,358.0
3,2023-01-03 06:30:00,2023,1,1,3,Tuesday,06:30,726.0,84.0,68.0,878.0,580.8,84.0,170.0,835.0
4,2023-01-04 15:30:00,2023,1,1,4,Wednesday,15:30,371.0,118.0,92.0,581.0,296.8,118.0,230.0,645.0


In [11]:
df_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10662 entries, 0 to 10661
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   WAKTU     10662 non-null  datetime64[ns]
 1   TAHUN     10662 non-null  int64         
 2   BULAN     10662 non-null  int64         
 3   PEKAN     10662 non-null  int64         
 4   TANGGAL   10662 non-null  int64         
 5   HARI      10662 non-null  object        
 6   JAM       10662 non-null  object        
 7   MOTOR     10662 non-null  float64       
 8   MOBIL     10662 non-null  float64       
 9   TRUK/BUS  10662 non-null  float64       
 10  JUMLAH    10662 non-null  float64       
 11  SMP_MTR   10662 non-null  float64       
 12  SMP_MBL   10662 non-null  float64       
 13  SMP_TRK   10662 non-null  float64       
 14  SMP       10662 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int64(4), object(2)
memory usage: 1.2+ MB


In [12]:
df_compiled = menggabungkan_data(df, df_missing)
df_compiled.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 00:00:00,2023,1,1,1,Sunday,00:00,103.0,25.0,14.0,142.0,82.4,25.0,35.0,142.0
1,2023-01-01 00:15:00,2023,1,1,1,Sunday,00:15,111.0,16.0,12.0,139.0,88.8,16.0,30.0,135.0
2,2023-01-01 00:30:00,2023,1,1,1,Sunday,00:30,114.0,26.0,6.0,146.0,91.2,26.0,15.0,132.0
3,2023-01-01 00:45:00,2023,1,1,1,Sunday,00:45,76.0,24.0,11.0,111.0,60.8,24.0,27.5,112.0
4,2023-01-01 01:00:00,2023,1,1,1,Sunday,01:00,73.0,24.0,7.0,104.0,58.4,24.0,17.5,100.0


In [13]:
df_compiled.to_excel('../Data/Preprocessing/Data Balonggandu/2023/2_input_balonggandu_2023.xlsx', index=False)

In [14]:
len(df_compiled)

35040